# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [1]:
import numpy as np

In [2]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [3]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.074020908411283, 10.0)

In [4]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    no_defaut = sum(default_time > mat)*np.exp(-rf_rate*mat)
    defaut = sum(np.exp(-rf_rate*default_time[default_time < mat]) * recovery)
    return (no_defaut + defaut)/ n_sample

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)

# Find the mean and std by calling the function 100 times. 
corp_values = []
for i in range(100):
     corp_values.append(corp_bond(mat, def_rate, rf_rate, recovery, n_sample))
print("mean :", np.mean(corp_values))
print("standard deviation :", np.std(corp_values))


mean : 0.4401601964025981
standard deviation : 0.002361120816381905


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [5]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=round(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999996, 10.0)

In [6]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    ### <--
    assert n_sample%2 == 0, "n_sample must be even for antithetic method"
    if antithetic and not mean_match:
        U = np.random.uniform(size=round(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    elif mean_match and not antithetic:
        U = np.random.uniform(size=round(n_sample))
        default_time = -(1/def_rate)*np.log(U)
        default_time += 1/def_rate-default_time.mean()
    elif mean_match and antithetic:
        U = np.random.uniform(size=round(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        default_time += 1/def_rate-default_time.mean()
    else:
        U = np.random.uniform(size=round(n_sample))
        default_time = -(1/def_rate)*np.log(U)
    
    no_defaut = sum(default_time > mat)*np.exp(-rf_rate*mat)
    defaut = sum(np.exp(-rf_rate*default_time[default_time < mat]) * recovery)    
    return (no_defaut + defaut)/ n_sample

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
anti_values, mean_values, both_values = [], [], []
for i in range(100):
    anti_values.append(corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=False))
    mean_values.append(corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=False, mean_match=True))
    both_values.append(corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=True))

for lst, method in zip([anti_values, mean_values, both_values], ['antithetic', 'mean match', 'both']):
    print(method, "\n", "mean :", np.mean(lst), "\n", "standard deviation :", np.std(lst))

antithetic 
 mean : 0.44013326826775206 
 standard deviation : 0.0018041991059442393
mean match 
 mean : 0.44028722639667 
 standard deviation : 0.0016075157803874223
both 
 mean : 0.4405095917510037 
 standard deviation : 0.001609805012397031


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

### Put the analytic expression for the corporate bond price

$Price = e^{-10r}F(T>10) + \int_0^{10} 0.3e^{-tr}P(t)\, dt$
=0.3*0.1/0.13 + np.exp(-1.3)*(1-0.3*10/13) = 0.44040907156462505

All MC results are fairly close the analitical results, while the antithetic and mean-matched MC method defitinely delivers the most stable simulating results.